## Blind docking with qvina-w and GaMD simulations by Yeng-Tseng wang (c00jsw00@gmail.com)

## Step 1: Dock the ligand with qvina-w (bash shell only in the block)

In [5]:
!source  /usr/local/mambaforge/amber.sh
!cp *.pdb rec.pdb
!cp *.mol dock.mol
!rm -rf *.mol2
!rm -rf *.pdbqt
!obabel -imol dock.mol -omol2 -O dock.mol2 --gen3d -h
!antechamber -i dock.mol2 -fi mol2 -o ligand.mol2 -fo mol2
!/usr/local/mgltools/bin/python2.7 /usr/local/mgltools/MGLToolsPckgs/AutoDockTools/Utilities24/prepare_ligand4.py -l ligand.mol2 -o ligand.pdbqt
!/usr/local/mgltools/bin/python2.7 /usr/local/mgltools/MGLToolsPckgs/AutoDockTools/Utilities24/prepare_receptor4.py -r rec.pdb -o rec.pdbqt 
!git clone https://github.com/QVina/qvina.git
!cp qvina/bin/qvina-w .
!chmod u+x qvina-w
!./qvina-w --config conf --ligand ligand.pdbqt --out dock-out.pdbqt --log dock-out.txt

cp: target 'rec.pdb' is not a directory
cp: target 'dock.mol' is not a directory
1 molecule converted

Welcome to antechamber 22.0: molecular input file processor.

Info: acdoctor mode is on: check and diagnose problems in the input file.
Info: The atom type is set to gaff; the options available to the -at flag are
      gaff, gaff2, amber, bcc, and sybyl.

-- Check Format for mol2 File --
   Status: pass
-- Check Unusual Elements --
   Status: pass
-- Check Open Valences --
         the connectivity (2) for atom type (N.ar) defined in CORR_NAME_TYPE.DAT.
But, you may safely ignore the warnings if your molecule
         uses atom names or element names as atom types.
-- Check Geometry --
      for those bonded   
      for those not bonded   
   Status: pass
-- Check Weird Bonds --
   Status: pass
-- Check Number of Units --
   Status: pass
acdoctor mode has completed checking the input file.


adding gasteiger charges to peptide
fatal: destination path 'qvina' already exists and is no

pdb viwer

In [6]:
import py3Dmol

def drawit2(m,confId=-1):
    mb = Chem.MolToMolBlock(m,confId=confId)
    p = py3Dmol.view(width=400, height=400)
    p.addModel(mb,'sdf')
    p.setStyle({'stick':{}})
    p.setBackgroundColor('0xeeeeee')
    p.zoomTo()
    return p

def DrawComplex(protein,ligand):
    complex_pl = Chem.MolToPDBBlock(Chem.CombineMols(protein,ligand))
    #complex_mol=Chem.CombineMols(receptor,mols[-1])
    view = py3Dmol.view(width=600,height=600)
    view.addModel(complex_pl,'pdb')
    #view.addModel(Chem.MolToMolBlock(mols[0]),'sdf')
    chA = {'chain':['A','B']}
    chB = {'resn':'UNL'}
    view.setStyle(chA,{'cartoon': {'color':'green'}})
    #view.setStyle(chA,{'lines': {}})
    view.addStyle({'chain':'A','resi':[116,140,200]},{'stick':{'colorscheme':'skyblueCarbon'}}) ##reaction center
    view.addStyle({'chain':'A','resi':[136,138,139,148,150,153,154,157,183,211,214,200]},{'cartoon': {'color':'yellow'}}) ##allosteric site
    view.addSurface(py3Dmol.VDW,{'opacity':0.8}, chB)
    view.setStyle(chB,{'stick':{}})
    view.zoomTo()
    return view   


In [7]:
from rdkit import Chem

### Convert docked ligands from PDBQT to SDF

In [8]:
!/usr/local/mgltools/bin/python2.7 /usr/local/mgltools/MGLToolsPckgs/AutoDockTools/Utilities24/pdbqt_to_pdb.py -f dock-out.pdbqt -o dock-out.pdb

In [9]:
#!mols = [m for m in Chem.MolFromPDBFile('dock-out.pdb') if m is not None]
lig = Chem.MolFromPDBFile('dock-out.pdb')

In [10]:
#drawit2(mols[0])
drawit2(lig)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## Step 5: Visualize the docked Protein-Ligand Complex

In [16]:
receptor = Chem.MolFromPDBFile('rec.pdb')
DrawComplex(receptor,lig)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

ambertools: top and crd files

In [11]:
!antechamber -fi mol2 -fo prepi -i dock.mol2 -o dock.prepi -c bcc -j 4 -at gaff
!parmchk2 -i dock.prepi -o dock.frcmod -f prepi
##leaprc.water.tip3p
!rm -rf tleapin

with open("tleapin", "w") as output:
    output.write('source /home/c00jsw00/jupyter/cmd/leaprc.protein.ff14SB'+'\n')
    output.write('source /home/c00jsw00/jupyter/cmd/leaprc.gaff'+'\n')
    output.write('source /home/c00jsw00/jupyter/cmd/leaprc.water.tip3p'+'\n')
    output.write('loadamberprep dock.prepi'+'\n')
    output.write('mod = loadAmberParams dock.frcmod'+'\n')
    output.write('set UNL head null'+'\n')
    output.write('set UNL tail null'+'\n')
    output.write('aa = loadpdb dock-out.pdb '+'\n')
    output.write('bb = loadpdb rec.pdb '+'\n')
    output.write('cc = combine {aa bb}'+'\n')
    output.write('saveAmberParm cc  z.top z.crd'+'\n')
    output.write('addions cc Cl- 0'+'\n')
    output.write('addions cc Na+ 0'+'\n')
    output.write('solvatebox cc TIP3PBOX 20.0'+'\n')
    output.write('saveAmberParm cc  zw.top zw.crd'+'\n')
    output.write('quit'+'\n')

!tleap -s -f tleapin
!ls -l


Welcome to antechamber 22.0: molecular input file processor.

Info: acdoctor mode is on: check and diagnose problems in the input file.
Info: The atom type is set to gaff; the options available to the -at flag are
      gaff, gaff2, amber, bcc, and sybyl.

-- Check Format for mol2 File --
   Status: pass
-- Check Unusual Elements --
   Status: pass
-- Check Open Valences --
         the connectivity (2) for atom type (N.ar) defined in CORR_NAME_TYPE.DAT.
But, you may safely ignore the warnings if your molecule
         uses atom names or element names as atom types.
-- Check Geometry --
      for those bonded   
      for those not bonded   
   Status: pass
-- Check Weird Bonds --
   Status: pass
-- Check Number of Units --
   Status: pass
acdoctor mode has completed checking the input file.

Info: Total number of electrons: 202; net charge: 0

Running: /usr/local/mambaforge/bin/sqm -O -i sqm.in -o sqm.out

-I: Adding /usr/local/mambaforge/dat/leap/prep to search path.
-I: Adding /usr

OPENMM cMD simulations for 5000000 steps

In [2]:
import sys
sys.path.append("/usr/local/mambaforge/lib/python3.10/site-packages")
from simtk.openmm import *
from simtk.openmm.app import *
from simtk.unit import *
from mdtraj.reporters import NetCDFReporter
from sys import stdout

# Input Files

prmtop = AmberPrmtopFile('zw.top')
inpcrd = AmberInpcrdFile('zw.crd')

# System Configuration

nonbondedMethod = PME
nonbondedCutoff = 1.0*nanometers
ewaldErrorTolerance = 0.0005
constraints = HBonds
rigidWater = True
constraintTolerance = 0.000001
hydrogenMass = 1.5*amu

# Integration Options

dt = 0.004*picoseconds
temperature = 300*kelvin
friction = 1.0/picosecond
pressure = 1.0*atmospheres
barostatInterval = 25

# Simulation Options

steps = 50000
equilibrationSteps = 1000
platform = Platform.getPlatformByName('CUDA')
platformProperties = {'Precision': 'mixed'}
dcdReporter = DCDReporter('trajectory.dcd', 1000)
dataReporter = StateDataReporter('log.txt', 1000, totalSteps=steps,
    step=True, time=True, speed=True, progress=True, potentialEnergy=True, temperature=True, density=True, separator='\t')
checkpointReporter = CheckpointReporter('checkpoint.chk', 1000)

# Prepare the Simulation

print('Building system...')
topology = prmtop.topology
positions = inpcrd.positions
system = prmtop.createSystem(nonbondedMethod=nonbondedMethod, nonbondedCutoff=nonbondedCutoff,
    constraints=constraints, rigidWater=rigidWater, ewaldErrorTolerance=ewaldErrorTolerance, hydrogenMass=hydrogenMass)
system.addForce(MonteCarloBarostat(pressure, temperature, barostatInterval))
integrator = LangevinMiddleIntegrator(temperature, friction, dt)
integrator.setConstraintTolerance(constraintTolerance)
simulation = Simulation(topology, system, integrator, platform)
simulation.context.setPositions(positions)
if inpcrd.boxVectors is not None:
    simulation.context.setPeriodicBoxVectors(*inpcrd.boxVectors)

# Write XML serialized objects

with open("system.xml", mode="w") as file:
    file.write(XmlSerializer.serialize(system))
with open("integrator.xml", mode="w") as file:
    file.write(XmlSerializer.serialize(integrator))

# Minimize and Equilibrate

print('Performing energy minimization...')
simulation.minimizeEnergy()
print('Equilibrating...')
simulation.context.setVelocitiesToTemperature(temperature)
simulation.step(equilibrationSteps)

# Simulate

print('Simulating...')
simulation.reporters.append(
        NetCDFReporter('amber.nc', 100)
)
simulation.reporters.append(dcdReporter)
simulation.reporters.append(dataReporter)
simulation.reporters.append(checkpointReporter)
simulation.currentStep = 0
simulation.reporters.append(NetCDFReporter('amber.nc', 1000))   # <-- AMBER compatible
simulation.step(steps)

# Write file with final simulation state

simulation.saveState("final_state.xml")
print('Simulating end...')

Building system...
Performing energy minimization...
Equilibrating...
Simulating...
Simulating end...


In [3]:
!cpptraj -i cpptraj


CPPTRAJ: Trajectory Analysis. V6.4.4 (AmberTools)
    ___  ___  ___  ___
     | \/ | \/ | \/ | 
    _|_/\_|_/\_|_/\_|_

| Date/time: 03/07/23 17:12:35
| Available memory: 42.780 GB

INPUT: Reading input from 'cpptraj'
  [parm zw.top]
	Reading 'zw.top' as Amber Topology
	Radius Set: modified Bondi radii (mbondi)
  [trajin amber.nc 50 50]
	Reading 'amber.nc' as Amber NetCDF
  [trajout equ11.rst restart]
	Writing 'equ11.rst' as Amber Restart
---------- RUN BEGIN -------------------------------------------------

PARAMETER FILES (1 total):
 0: zw.top, 63699 atoms, 20578 res, box: Orthorhombic, 20422 mol, 20416 solvent

INPUT TRAJECTORIES (1 total):
 0: 'amber.nc' is a NetCDF AMBER trajectory with coordinates, time, box, Parm zw.top (Orthorhombic box) (reading 1 of 50)
  Coordinate processing will occur on 1 frames.

OUTPUT TRAJECTORIES (1 total):
  'equ11.rst' (1 frames) is an AMBER restart file

BEGIN TRAJECTORY PROCESSING:
.....................................................
ACTIVE OUT

In [ ]:
print('GaMD Simulating begin...')
!gamdRunner xml  zwmdcrd-01.xml
print('GaMD Simulating end...')

GaMD Simulating begin...
Running: 	  15500000  steps
Start Time: 	 Mar-07-2023    17:17:26
End Time: 	 Mar-07-2023    23:43:15
Execution rate for this run:   669.5753596267657  steps per second.
Daily execution rate:          115.7026221435051  ns per day.
